## PPC

In [1]:
import sys
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
import pandas as pd
import torch

import scvi
from scvi.dataset import GeneExpressionDataset, CellMeasurement, AnnDatasetFromAnnData
from scvi.models import VAE, TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer, Posterior, UnsupervisedTrainer

import anndata
import scanpy as sc
import umap
import sparse

sns.set(context="notebook", font_scale=1.3, style="ticks")
save_path = "/data/yosef2/users/adamgayoso/projects/totalVI_journal/data/"
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
%load_ext autoreload
%autoreload 2
%matplotlib inline
overwrite=False

sys.path.append("../utils/")
from totalppc import TotalPosteriorPredictiveCheck as totalPPC
from scvi import set_seed

# colors = ["#9b59b6", "#3498db", "#95a5a6", "#34495e", "#e74c3c", "#2ecc71"]
# colors_3 = ["#9b59b6", "#95a5a6", "#34495e", "#e74c3c", "#2ecc71"]

colors = ["#3B7EA1", "#FDB515", "#D9661F", "#859438", "#EE1F60", "#00A598"]
colors_3 = ["#3B7EA1", "#D9661F", "#859438", "#EE1F60", "#00A598"]

set_seed(0)

In [2]:
colors = ["#3B7EA1", "#FDB515", "#D9661F", "#859438", "#EE1F60", "#00A598"]
sns.set(context="notebook", font_scale=1.3, style="ticks")
sns.set_palette(sns.color_palette(colors))
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['savefig.transparent'] = True
# plt.rcParams['figure.figsize'] = (4, 4)

sc.settings._vector_friendly = True
DPI = 300
W_SPACE = 0.18

In [3]:
anndataset_pbmc = anndata.read(save_path + "pbmc_10k_protein_v3.h5ad")

In [4]:
dataset_pbmc = AnnDatasetFromAnnData(ad=anndataset_pbmc)
protein_data = CellMeasurement(
    name="protein_expression",
    data=anndataset_pbmc.obsm["protein_expression"].astype(np.float32),
    columns_attr_name="protein_names",
    columns=anndataset_pbmc.uns["protein_names"],
)
dataset_pbmc.initialize_cell_measurement(protein_data)

datasets = [dataset_pbmc]

[2020-09-20 22:30:11,859] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-09-20 22:30:11,862] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-09-20 22:30:12,043] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 22:30:12,225] INFO - scvi.dataset.dataset | Downsampled from 6855 to 6855 cells


In [5]:
for d in datasets:
    d.update_genes(d.var["highly_variable"])

[2020-09-20 22:30:12,265] INFO - scvi.dataset.dataset | Downsampling from 16727 to 4000 genes
[2020-09-20 22:30:12,384] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 22:30:12,474] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-09-20 22:30:12,565] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 22:30:12,610] INFO - scvi.dataset.dataset | Downsampled from 6855 to 6855 cells


In [6]:
use_cuda = True
lr = 4e-3
early_stopping_kwargs = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "patience": 45,
    "threshold": 0,
    "reduce_lr_on_plateau": True,
    "lr_patience": 30,
    "lr_factor": 0.6,
    "posterior_class": TotalPosterior,
}

In [ ]:
vaes = []
trainers = {}
posteriors = {}

latent_dims = [5, 10, 20, 100]
for n_latent in latent_dims:
    trainers[n_latent] = []
    posteriors[n_latent] = []
    for i in range(5):
        vae_pbmc = TOTALVI(
            dataset_pbmc.nb_genes, len(dataset_pbmc.protein_names), n_latent=n_latent,
        )
        trainer_pbmc = TotalTrainer(
            vae_pbmc,
            dataset_pbmc,
            train_size=0.80,
            test_size=0.05,
            use_cuda=use_cuda,
            frequency=1,
            data_loader_kwargs={"batch_size": 256, "pin_memory": False},
            early_stopping_kwargs=early_stopping_kwargs,
#             seed=i,
        )
        trainer_pbmc.train(lr=lr, n_epochs=500)
        trainers[n_latent].append(trainer_pbmc)
        posteriors[n_latent].append(
            trainer_pbmc.create_posterior(type_class=TotalPosterior)
        )
        vaes.append(vae_pbmc)


[2020-09-20 22:39:56,880] INFO - scvi.inference.trainer | Reducing LR on epoch 471.



[2020-09-20 22:57:37,937] INFO - scvi.inference.trainer | Reducing LR on epoch 395.
[2020-09-20 22:58:15,924] INFO - scvi.inference.trainer | Reducing LR on epoch 428.
[2020-09-20 22:59:19,442] INFO - scvi.inference.trainer | Reducing LR on epoch 483.



[2020-09-20 23:16:15,513] INFO - scvi.inference.trainer | Reducing LR on epoch 364.



[2020-09-20 23:25:11,576] INFO - scvi.inference.trainer | Reducing LR on epoch 329.
[2020-09-20 23:26:29,017] INFO - scvi.inference.trainer | Reducing LR on epoch 396.
[2020-09-20 23:27:17,352] INFO - scvi.inference.trainer | Reducing LR on epoch 438.
[2020-09-20 23:27:56,399] INFO - scvi.inference.trainer | Reducing LR on epoch 472.
[2020-09-20 23:28:13,683] INFO - scvi.inference.trainer | 
Stopping early: no improvement of more than 0 nats in 45 epochs
[2020-09-20 23:28:13,685] INFO - scvi.inference.trainer | If the early stopping criterion is too strong, please instantiate it with different parameters in the train method.


[2020-09-20 23:35:43,795] INFO - scvi.inference.trainer | Reducing LR on epoch 389.
[2020-09-20 23:37:02,214] INFO - scvi.inference.trainer | Reducing LR on epoch 457.



[2020-09-20 23:45:53,693] INFO - scvi.inference.trainer | Reducing LR on epoch 418.
[2020-09-20 23:46:35,057] INFO - scvi.inference.trainer | Reducing LR on epoch 454.



### Marginal LL

In [ ]:
reconst_df = pd.DataFrame(index=np.arange(5), columns=latent_dims)

for z_dim, t_list in trainers.items():
    i = 0
    for t in t_list:
        reconst_df.loc[i, z_dim] = t.validation_set.marginal_ll(n_mc_samples=5000)
#         reconst_df.loc[i, z_dim] = t.train_set.marginal_ll()
        i += 1

In [ ]:
reconst_df

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.boxplot(data=-reconst_df)#, y="Marginal LL", x="Dim(z)")
ax.set_xlabel("Number of latent dimensions")
ax.set_ylabel(r"$\log p(x, y)$")
sns.despine()
fig.savefig("figures/log_lik_stability.pdf", bbox_inches="tight")

# plt.tight_layout()
# plt.savefig("figures/rec_error.pdf")

### UMAP test

In [ ]:
latent_mean = posteriors[20][0].sequential().get_latent()[0]

In [ ]:
anndataset_pbmc.obsm["X_totalVI"] = latent_mean

In [ ]:
from numba import jit

@jit(nopython=True)
def hellinger(x, y):
    result = 0.0
    l1_norm_x = 0.0
    l1_norm_y = 0.0

    for i in range(x.shape[0]):
        result += np.sqrt(x[i] * y[i])
        l1_norm_x += x[i]
        l1_norm_y += y[i]

    if l1_norm_x == 0 and l1_norm_y == 0:
        return 0.0
    elif l1_norm_x == 0 or l1_norm_y == 0:
        return 1.0
    else:
        return np.sqrt(1 - result / np.sqrt(l1_norm_x * l1_norm_y))

sc.pp.neighbors(anndataset_pbmc, use_rep="X_totalVI", n_neighbors=25, metric=hellinger)
sc.tl.umap(anndataset_pbmc, min_dist=0.3, n_components=2)
sc.tl.leiden(anndataset_pbmc, key_added="leiden_totalVI", resolution=0.7)

In [ ]:
fig = sc.pl.umap(
    anndataset_pbmc, 
    color=["leiden_totalVI"],
    return_fig=True,
    frameon=False
)


### Denoising stability

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


n_samples = 25


foreground_prob = {}

for z_dim, p_list in posteriors.items():
    i = 0
    foreground_prob[z_dim] = []
    for post in p_list:
        # Probability of background
        py_mixing = np.mean(
            sigmoid(
                post.sequential().get_sample_mixing(
                    n_samples=n_samples, give_mean=False,
                )
            ),
            axis=-1,
        )
        foreground_prob[z_dim].append(1 - py_mixing)

In [ ]:
ground_truth = foreground_prob[10][0]
ground_truth.shape

In [ ]:
from scipy.stats import spearmanr, pearsonr

corrs = {}
bad_pros = []
for z_dim, fp_list in foreground_prob.items():
    corrs[z_dim] = []
    for fp in fp_list:
        for j in range(fp.shape[1]):
            corr, _ = pearsonr(fp[:, j], ground_truth[:, j])
            if corr < 0.8:
                bad_pros.append(anndataset_pbmc.uns["protein_names"][j])
            corrs[z_dim].append(corr)

In [ ]:
import collections

collections.Counter(bad_pros)

In [ ]:
protein_df = pd.DataFrame(
    np.log1p(anndataset_pbmc.obsm["protein_expression"].copy()),
    columns=anndataset_pbmc.uns["protein_names"],
)

In [ ]:
plt.hist(protein_df["CD15_TotalSeqB"], bins=20)

In [ ]:
df = pd.DataFrame(corrs)
fig, ax = plt.subplots()
sns.boxplot(data=df)
ax.set_xlabel("Number of latent dimensions")
ax.set_ylabel("Pearson correlation of " + r"$\pi_{nt}$")
sns.despine()
fig.savefig("figures/denoising_stability.pdf", bbox_inches="tight")

In [ ]:
test=1